# Proyecto 2
 
- *Dayana Valentina Gonzalez Vargas*
- *Juan Manuel Ramirez*

## Librerias

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

### Cargar los Datos

In [55]:
df=pd.read_csv('emotions_by_voice_registers.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,X,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0,1,1,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,1,2,2,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,2,3,3,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,4,5,5,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,5,6,6,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  909 non-null    int64  
 1   Unnamed: 0    909 non-null    int64  
 2   X             909 non-null    int64  
 3   meanfreq      909 non-null    float64
 4   sd            909 non-null    float64
 5   median        909 non-null    float64
 6   Q25           909 non-null    float64
 7   Q75           909 non-null    float64
 8   IQR           909 non-null    float64
 9   skew          909 non-null    float64
 10  kurt          909 non-null    float64
 11  sp.ent        909 non-null    float64
 12  sfm           909 non-null    float64
 13  mode          909 non-null    float64
 14  centroid      909 non-null    float64
 15  meanfun       909 non-null    float64
 16  minfun        909 non-null    float64
 17  maxfun        909 non-null    float64
 18  meandom       909 non-null    

### Eliminación de datos que no son necesarios o son de tipo indíce.
Eliminaremos los datos del DataSet que son tipo indíce.

In [57]:
df = df.drop(columns = [ 'Unnamed: 0.1', 'Unnamed: 0', 'X'])
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,2.863717,0.923566,0.307220,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,3.878650,0.918848,0.298859,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,4.589995,0.919519,0.313069,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,3.680995,0.921361,0.329295,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,5.031744,0.926238,0.337047,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad


**Dividimos en dos, el primer conjuntos Y es el conjunto con la etiqueta y el segundo es el conjunto X que va a ser el conjunto de las variables independientes sin la etiqueta**

In [58]:
y = df['label']
y = pd.get_dummies( y, drop_first = True)
X = df.drop(columns =['label'])

### Crear los conjuntod de Train, Validación y Test.
A partir de la función *train_test_split* vamos a realizarla dos veces, la primera para dividir en test y train. La segunda para ese train anterior dividirlo en val y train.

In [61]:
X_train1, X_test, y_train1, y_test = train_test_split( X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split( X_train1, y_train1, test_size=0.20, random_state=42)

## Preparación de los datos

In [62]:
class MyDataset():
 
  def __init__(self,x,y):
    y=y.values
    X=x.values
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [63]:
train_sec=MyDataset(X_train,y_train)
test_sec=MyDataset(X_test,y_test)
val_sec=MyDataset(X_val,y_val)

In [64]:
train_data=DataLoader(
    train_sec,
    batch_size=1,
    shuffle=False,
 )

test_data=DataLoader(
    test_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

val_data=DataLoader(
    val_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [65]:
for i, (data, labels) in enumerate(test_data):
  print(data.shape, labels.shape)
  print(data,labels)
  break;

torch.Size([3, 20]) torch.Size([3, 2])
tensor([[0.2276, 0.0391, 0.2321, 0.2119, 0.2580, 0.0461, 2.3979, 9.7464, 0.8642,
         0.1852, 0.2336, 0.2276, 0.1917, 0.0584, 0.2759, 0.8516, 0.1484, 4.8438,
         4.6953, 0.1747],
        [0.2048, 0.0441, 0.2082, 0.1876, 0.2297, 0.0421, 1.9322, 6.3588, 0.8904,
         0.3525, 0.2066, 0.2048, 0.1781, 0.0343, 0.2462, 0.6719, 0.0781, 4.4219,
         4.3438, 0.1271],
        [0.1672, 0.0607, 0.1612, 0.1167, 0.2118, 0.0951, 1.0096, 3.4679, 0.9367,
         0.4687, 0.0938, 0.1672, 0.1152, 0.0208, 0.2222, 0.5907, 0.0859, 5.3203,
         5.2344, 0.1471]]) tensor([[1., 0.],
        [1., 0.],
        [0., 1.]])


## Definir la clase NET 


Se genera la clase NET para construir la red neuronal. Urilizamos 2 capas ocultas para dar mayor adaptabilidad al patron de los datos. Los nodos se establecieron como una reduccion uniforme desde la cantidad de entrada hasta la cantidad de salida. Como funcion de activacion tomamos la tangente hipebolica con el fin de tener una convergencia mas suave alejandonos de casos de sobre entrenamiento.

In [66]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20,14)
        self.fc2 = nn.Linear(14,8)
        self.fc3 = nn.Linear(8,2)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return x

Para el aprendizaje y evaluacion del modelo se utilizaran las funciones de decenso del gradiente y validacion cruzada en entropia. Se encontro que el decenso del gradiente es la mejor funcion 

In [67]:
model=Net()

optimizer= torch.optim.SGD(model.parameters(), lr=0.1)
criterion= nn.CrossEntropyLoss()

**Entrenar el modelo e ir imprimiendo los errores del test y la validación.**

In [68]:
def train_model(model,optimizer,loss_module,train_loader,valid_loader,num_epochs):
  
  valid_loss_min =np.inf  #Vamos a encontrar el menor valor de error de validación. Por eso la inicializmaos como 'infinito'
  
  for i in range(num_epochs):
    model.train()  #ponemos el modelo en modo entrenamiento. Es importante en otras arquitecturas como redes convolucionales.
    train_loss = 0.0
    v_loss = 0.0

    for data, target in train_loader:
         # Reiniciar los gradientes
        optimizer.zero_grad()

        # forward pass: calcular la salida para los datos de entrada..
        preds = model(data)
        preds = preds.squeeze(dim=1)

        # calculate the batch loss
        loss = loss_module(preds, target)
        
        # backpropagation: cálculo de gradientes
        loss.backward()
   
        # actualizar los parámetros
        optimizer.step()

        # actualizar la cuenta de costos a lo largo de los lotes
        train_loss += loss.item()*data.size(0)
    # for data,labels in testloader:

    train_loss = train_loss/len(train_loader.dataset) 

    model.eval() #Ponemos el modelo en modo evaluación.
    
    for data,target in valid_loader:
      output=model(data)
      valid_loss= criterion(output, target)
      valid_loss += loss.item()*data.size(0)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    #imprimir estadísticas de entrenamiento y validación
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        i, train_loss, valid_loss))
    

    #Guardamos el modelo con el menor error de validación.
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_bikeshare.pt')
        valid_loss_min = valid_loss
    

In [69]:
train_model(model, optimizer,criterion,train_data,val_data,50)

Epoch: 0 	Training Loss: 0.477401 	Validation Loss: 0.002290
Validation loss decreased (inf --> 0.002290).  Saving model ...
Epoch: 1 	Training Loss: 0.475920 	Validation Loss: 0.002278
Validation loss decreased (0.002290 --> 0.002278).  Saving model ...
Epoch: 2 	Training Loss: 0.474709 	Validation Loss: 0.002273
Validation loss decreased (0.002278 --> 0.002273).  Saving model ...
Epoch: 3 	Training Loss: 0.475487 	Validation Loss: 0.002274
Epoch: 4 	Training Loss: 0.475564 	Validation Loss: 0.002251
Validation loss decreased (0.002273 --> 0.002251).  Saving model ...
Epoch: 5 	Training Loss: 0.475435 	Validation Loss: 0.002253
Epoch: 6 	Training Loss: 0.478216 	Validation Loss: 0.002251
Epoch: 7 	Training Loss: 0.476768 	Validation Loss: 0.002260
Epoch: 8 	Training Loss: 0.476634 	Validation Loss: 0.002245
Validation loss decreased (0.002251 --> 0.002245).  Saving model ...
Epoch: 9 	Training Loss: 0.476140 	Validation Loss: 0.002252
Epoch: 10 	Training Loss: 0.476721 	Validation Los

**Visualizamos los parametros del mejor modelo**

In [70]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc1.weight tensor([[ 7.4574e-02,  2.0312e-01,  2.1750e-01, -1.3612e-01,  2.9779e-01,
          2.7565e-01, -1.6340e-01, -1.9764e+00,  1.3271e+00,  1.0451e+00,
          1.7439e-01, -1.1671e-01,  1.7104e-01,  1.2991e-01,  3.8204e-01,
         -7.6393e-01, -5.1842e-02,  1.0509e+00,  1.2195e+00, -1.7928e-01],
        [ 5.2211e-02, -1.9920e-01, -1.2577e-01,  9.1154e-02,  1.6469e-01,
          1.7827e-01, -1.2934e-01,  2.4912e-01, -1.6447e-01,  5.9074e-02,
          1.9445e-01, -4.4041e-02,  9.5997e-02, -1.6958e-02, -3.5822e-02,
          2.0804e-01,  2.6692e-01, -3.0354e-02, -3.2887e-01,  2.0192e-01],
        [-6.7945e-02, -2.0129e-01, -6.6339e-02,  8.1698e-02, -2.3795e-02,
          3.2022e-02, -3.2879e-02, -2.4122e-01, -2.0079e-01, -1.8763e-01,
          7.4315e-02, -3.6408e-02, -1.5757e-01, -1.4638e-01,  8.2392e-02,
          1.3684e-01,  1.4555e-01,  1.7231e-01,  2.4622e-01, -1.0315e-01],
        [-8.5410e-02,  1.1774e-01,  7.0145e-02, -1.4409e-01,  3.1980e-02,
          1.5980e-03, -4

In [77]:
model.eval()
total, correct =0,0
for i, data in enumerate(test_data, 0):
    inputs, labels = data[0], data[1]
    
    # Put each image into a vector
    inputs = inputs.view(-1, 784)
    
    # Do the forward pass and get the predictions
    outputs = model(inputs)
    
    _, outputs = torch.max(outputs.data, 1) #mayor valor entre los dígitos.
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

RuntimeError: shape '[-1, 784]' is invalid for input of size 60

### Escoger una muestra aleatoria del DataSet
Vamos a tomar una muestra aleatoria con la función Sample y utilizar el modelo anterior para poder predecir el semintimiento.

In [73]:
df_all = pd.concat([X,y],axis = 1)

sample = df_all.sample(1)
sample.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,happy,sad
524,0.229763,0.052751,0.246154,0.215128,0.265385,0.050256,2.859533,12.289489,0.871022,0.333922,...,0.186087,0.026891,0.262295,1.329167,0.34375,6.804688,6.460938,0.218518,0,0


In [75]:
sample_y = df_all[['happy','sad']]
sample_x = df_all.drop(columns = ['happy','sad'])

In [ ]:
sample_sec=MyDataset(sample_x, sample_y)


sample_data=DataLoader(
    sample_sec,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

criterion= nn.MSELoss()
for data, target in sample_data:
  output=model(data)

print((output))
print((sample[['happy','sad']])